To begin with this exploratory data analysis of Tesla Stock, we import the needed libraries.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Once we have done this, we can start importing the required data from Yahoo Finance.

In [ ]:
tesla_df = yf.download("TSLA", period='5y')

# We are ignoring the double index as we know we are only working with tesla stocks, therefore we can remove the ticker from the data frame
tesla_df.columns = tesla_df.columns.get_level_values(0)

In [ ]:
print(tesla_df.head())
print("\n")
print(tesla_df.tail())
print("\n")
print(tesla_df.info())
print("\n")
print(tesla_df.describe())

Using the previous commands, we can already see some interesting information regarding the downloaded data, which is a data frame of Tesla Stock prices from the last 5 years. We have the open and close prices of the stock for every day in which the market is open (on weekends the market is closed). For each one of these days, we can see the lowest and highest price, as well as the volume traded that day.

The command .info() tells us the data frame counts with 1255 days of data from the last 5 years: Prices are given as floats and volume is given as an integer. Moreover, the data frame uses 58.8 KB of memory. One important thing to note is that we do not have any single null data from the 1255 days, which may facilitate data treatment.

To end with this section, we used .describe(), which gives us some statistical values of each data set (column) of the data frame, such as the mean, the standart deviation and so on. For example, the lowest price Tesla Stock had in the last 5 years was 101.809998 USD, and the highest was 488.540009 USD. This gives us a range of prices of nearly 400 USD (this is just a rough estimation, detailed results will be displayed throughout the curse of this notebook).

In [ ]:
tesla_df['returns'] = tesla_df['Close'].pct_change()
tesla_df['VOLAT30'] = tesla_df['returns'].rolling(window=30).std()
tesla_df['MA50'] = tesla_df['Close'].rolling(window=50).mean()
tesla_df['MA200'] = tesla_df['Close'].rolling(window=200).mean()

To analyze momentum and trend, we engineer several key technical indicators: daily returns, 30-day rolling volatility, and the 50/200-day Simple Moving Averages (SMAs).

Firstly, we added the daily returns, i.e. the daily changes in price. After that, we added a volatility indicator, which is calculated as the standard deviation of the 30 previous returns. As a last indicator, we have chosen the moving averages, the price average of the 50 and 200 precious days respectively.

Now let's illustrate this indicators in a variety of graphs.

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(tesla_df['Close'], label='TSLA close price', color='red')
plt.plot(tesla_df['MA50'], label='50 days Moving Average', color='blue')
plt.plot(tesla_df['MA200'], label='200 days Moving Average', color='orange')

# Creating a column with the difference of both means
tesla_df['diff'] = tesla_df['MA50'] - tesla_df['MA200']

# Detecting crosses (when 'diff' sign changes)
tesla_df['signal'] = np.sign(tesla_df['diff'])
tesla_df['cross'] = tesla_df['signal'].diff()

# Golden Cross: 
golden_crosses = tesla_df[tesla_df['cross'] == 2]

# Death Cross: 
death_crosses = tesla_df[tesla_df['cross'] == -2]

# Adding the crosses to the graph
plt.scatter(golden_crosses.index, golden_crosses['MA50'],
            color='gold', label='Golden Cross', s=100, zorder=5)

plt.scatter(death_crosses.index, death_crosses['MA50'],
            color='black', label='Death Cross', s=100, zorder=5)

plt.title(f'Close price and 50/200 Days Moving Averages for TSLA')
plt.xlabel('Date')
plt.ylabel('Price (USD)')
plt.legend()  
plt.grid(True) 
plt.show()

This chart also shows us when there is a golden cross (MA50 goes over MA200) as a yellow dot or a death cross (MA50 goes under MA200) as a black dot. A very common practice is to identify a golden cross as a buy signal and a death cross as a sell signal. Nonetheless, while these signals are traditionally interpreted as direct buy/sell triggers, an alternative strategic application is to use them to inform accumulation (DCA) strategies. A Death Cross, for instance, could signal the start of a period of price weakness potentially suitable for accumulation, while a Golden Cross could indicate a period of strength suitable for profit-taking or reducing positions.

In [ ]:
plt.figure(figsize=(14, 7))
#plt.bar(tesla_df.index, tesla_df['Volume'], color='skyblue', edgecolor='skyblue')
plt.fill_between(tesla_df.index, tesla_df['Volume'], color='skyblue')

plt.title('Tesla Stocks Volume Traded', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Volume', fontsize=12)
plt.tight_layout()
plt.show()

We can now see in this chart the historical volume traded for Tesla stocks. It is important to note that the volume comes in terms of stocks and not USD. Also note that the y axis is divided by ten to the eighth power.

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(tesla_df['VOLAT30'], label='TSLA 30 days Volatilty')

plt.title(f'Tesla Stocks 30 days volatility')
plt.xlabel('Date')
plt.ylabel('Volatility Value')
plt.grid(True) 
plt.show()

This is a simple graph where the volatility is displayed. This indicator is also used for a widely range of signals, as it indicates agressive spikes of the prices out of the common behaviour of the chart.

In [ ]:
sns.set_theme(style="whitegrid")

plt.figure(figsize=(10, 6))

barplot = sns.histplot(x='returns', data=tesla_df, kde=True)

# Usamos Matplotlib para añadir los toques finales y profesionales
plt.title('TSLA daily Returns', fontsize=16)
plt.xlabel('Values', fontsize=12)
plt.ylabel('Frequency', fontsize=12)

plt.tight_layout() # Ajusta el gráfico para que todo encaje bien
plt.show() # Muestra el gráfico final

In this final chart, we spot a normal distributions of Tesla Stock daily returns. We can see the mean is a value slightly higher than 0%, which indicates the bullish historical price action of Tesla.

As a conclusion, it is clear that TSLA is a very volatile asset, affected severely by 2023 rates hike and some political tensions between Elon and Donald Trump early this year. The bullish long term tendency of it along with all the innovations this company is pushing (such as Optimus as their robotical innovation or AI implementations), makes it an interesting asset to invest in. We could take advantage of its high volatility to accumulate stocks at very underrated prices and hold for mid-long term, leading to huge returns with low risk based on its fundamentals.